# geDIG Attention Update (Modern LLMs)

Goal: sample attention maps from newer LLMs (Llama 3.0, Llama 3.1, Phi-3) and check whether geDIG F keeps the same sign/delta vs. random.

Notes:
- Llama 3 requires an HF token and license acceptance (per model).
- Phi-3 is open and should run without a token.
- The notebook auto-selects dtype based on GPU (A100 -> bfloat16, T4 -> float16).


In [ ]:
from google.colab import drive

drive.mount('/content/drive')


In [ ]:
import os
import subprocess

if not os.path.isdir('InsightSpike-AI'):
    subprocess.check_call(['git', 'clone', 'https://github.com/miyauchikazuyoshi/InsightSpike-AI.git'])
%cd InsightSpike-AI
!pip -q install -U pip
!pip -q install transformers datasets accelerate sentencepiece networkx scipy huggingface_hub


In [ ]:
from huggingface_hub import login
from google.colab import userdata

hf_token = None
try:
    # Try to retrieve the token from Colab Secrets
    hf_token = userdata.get('HF_TOKEN')
    if hf_token:
        login(token=hf_token)
        print("Successfully logged in using Colab Secret 'HF_TOKEN'.")
    else:
        # Fallback to interactive login
        print("Secret 'HF_TOKEN' not found. Please log in interactively:")
        login()
except Exception:
    # If userdata is not available or fails, fallback to interactive
    print("Could not access Colab Secrets. Please log in interactively:")
    login()


In [ ]:
from huggingface_hub import hf_hub_download
from IPython.display import display, HTML

LLAMA3_MODELS = {
    'llama3': 'meta-llama/Meta-Llama-3-8B',
    'llama31': 'meta-llama/Llama-3.1-8B',
}
PHI3_ID = 'microsoft/Phi-3-mini-4k-instruct'

def show_status(ok: bool, title: str, detail: str = ''):
    color = '#1b8a5a' if ok else '#b02a37'
    msg = f'<b>{title}</b>' + (f'<br/>{detail}' if detail else '')
    display(HTML(f"<div style='border:1px solid {color};padding:10px;border-radius:6px;color:{color};margin:8px 0;'>{msg}</div>"))

llama3_ok = {}
token = hf_token if hf_token else None
for tag, model_id in LLAMA3_MODELS.items():
    label = 'Llama 3.0' if tag == 'llama3' else 'Llama 3.1'
    try:
        hf_hub_download(repo_id=model_id, filename='config.json', token=token)
        show_status(True, f'{label} download OK', f'Model: {model_id}')
        llama3_ok[tag] = True
    except Exception as exc:
        show_status(False, f'{label} download NOT available', f'{type(exc).__name__}: {exc}')
        llama3_ok[tag] = False
        print(f'If you need {label}, accept the license on HF and run login().')


In [ ]:
import subprocess

def _get_gpu_name():
    try:
        out = subprocess.check_output(
            ['nvidia-smi', '--query-gpu=name', '--format=csv,noheader'],
            text=True,
        ).strip()
        return out.splitlines()[0] if out else None
    except Exception:
        return None

GPU_NAME = _get_gpu_name()
if GPU_NAME:
    print('GPU:', GPU_NAME)
    if 'A100' in GPU_NAME or 'H100' in GPU_NAME:
        DTYPE = 'bfloat16'
    else:
        DTYPE = 'float16'
    DEVICE = 'cuda'
    DEVICE_MAP = 'auto'
else:
    print('GPU: none (CPU)')
    DTYPE = 'float32'
    DEVICE = 'cpu'
    DEVICE_MAP = None

ATTN_IMPL = 'eager'  # ensure attentions are returned
TEXT_COUNT = 16
LAYER_CAP = 4
ATTN_MAX_LEN = 256


In [ ]:
import json
import numpy as np
from pathlib import Path

outputs = {}

def run_model(model_id: str, tag: str):
    out_path = f'results/transformer_gedig/score_llm_{tag}.json'
    device_map_arg = f'--device-map {DEVICE_MAP}' if DEVICE_MAP else ''
    cmd = (
        f"python experiments/transformer_gedig/extract_and_score.py "
        f"--model '{model_id}' "
        f"--text-count {TEXT_COUNT} "
        f"--layer-cap {LAYER_CAP} "
        f"--attn-max-len {ATTN_MAX_LEN} "
        f"--percentile 0.90 "
        f"--out '{out_path}' "
        f"--device {DEVICE} --dtype {DTYPE} "
        f"{device_map_arg} "
        f"--trust-remote-code --attn-implementation {ATTN_IMPL}"
    )
    print(cmd)
    !{cmd}
    outputs[tag] = out_path

# Always run Phi-3
run_model(PHI3_ID, 'phi3')

# Run Llama 3 models only if access is available
for tag, model_id in LLAMA3_MODELS.items():
    if llama3_ok.get(tag):
        run_model(model_id, tag)
    else:
        print(f'Skipping {tag} run (no access).')


In [ ]:
def summarize(path):
    data = json.loads(Path(path).read_text(encoding='utf-8'))
    rows = [r for r in data if not r.get('subgraph')]
    if not rows:
        return None
    f_real = np.array([r['F'] for r in rows], dtype=float)
    f_rand = np.array([r.get('baseline_F_random') for r in rows], dtype=float)
    delta = f_real - f_rand
    return {
        'rows': len(rows),
        'F_real_mean': float(f_real.mean()),
        'F_rand_mean': float(f_rand.mean()),
        'delta_mean': float(delta.mean()),
        'delta_std': float(delta.std()),
        'delta_positive_ratio': float((delta > 0).mean()),
    }

summary = {tag: summarize(path) for tag, path in outputs.items()}
print(json.dumps(summary, indent=2))

summary_path = Path('results/transformer_gedig/score_llm_update.json')
summary_path.parent.mkdir(parents=True, exist_ok=True)
summary_path.write_text(json.dumps(summary, indent=2), encoding='utf-8')
print('Wrote:', summary_path)


In [ ]:
import shutil

save_dir = Path('/content/drive/MyDrive/insightspike/gedig_attention')
save_dir.mkdir(parents=True, exist_ok=True)

for tag, path in outputs.items():
    src = Path(path)
    if src.exists():
        shutil.copy2(src, save_dir / src.name)
        print('Saved:', save_dir / src.name)

summary_src = Path('results/transformer_gedig/score_llm_update.json')
if summary_src.exists():
    shutil.copy2(summary_src, save_dir / summary_src.name)
    print('Saved:', save_dir / summary_src.name)
